In [29]:
import pandas as pd
import numpy as np
from scipy.ndimage.interpolation import shift

In [12]:
df = pd.read_csv('price return monthly.csv')

In [13]:
df.head()

,Date,S&P 500 (TR),S&P 500 (Net TR),S&P 500
0,5/1/2013,2797.28,2582.38,1582.70
1,6/3/2013,2906.91,2681.48,1640.42
2,7/1/2013,2866.41,2642.84,1614.96
3,8/1/2013,3033.59,2795.86,1706.87
4,9/3/2013,2921.23,2690.40,1639.77


In [19]:
x = np.array(df['S&P 500'])

In [25]:
out = np.diff(x, axis=0)
df['price return'] = pd.DataFrame(out)

In [38]:
shift(out, 1, cval=0)

array([   0.  ,   57.72,  -25.46,   91.91,  -67.1 ,   55.23,   66.64,
         39.26,   31.08,  -90.09,  103.84,   39.79,   -1.84,   41.29,
         48.35,  -48.17,    6.  ,   55.01,   31.65,   35.63,    4.76,
        -37.35,   96.54,  -57.7 ,   48.6 ,  103.44,  -34.31,  -79.38,
         -9.19, -165.03,  180.23,   18.58,    7.03, -190.28,   38.97,
         94.43,    8.65,   17.9 ,   71.51,  -67.89,   70.91,  -12.66,
        -49.48,   79.36,   16.75,   71.72,  116.41,  -37.12,   29.49,
         41.73,  -11.05,   31.  ,    6.54,   72.57,   50.24,   62.86])

In [39]:
df['price return'] = pd.DataFrame(shift(out, 1, cval=0))


In [68]:
df['period']=df['price return'].apply(lambda x: 0 if (x < 20.00) & (x > -20.00) else 1)

In [78]:
def drop_columns(df, *args):
    '''
    Takes in a dataframe and a list of column names to be removed.
    Useful for when feeding into a model that does not take in datetime or labels.
    Returns the mutated dataframe with the desired columns removed.
    '''
    
    df.drop(list(args), axis=1, inplace=True)
    
    return df

def model_inputs(df, sector_column):
    '''
    Takes in the dataframe and the name of the column that describes the sectors.
    Returns x and y values as float32 types to be fed into the keras model. 
    There must not be any str or datetime types in any of the columns.
    '''
    
    temp_df = df.drop(sector_column, axis=1)
    x = temp_df.values
    x = x.astype(np.float32)
    
    y = df[sector_column].values.astype(np.float32)
    y = y.reshape(-1, 1)
    y = to_categorical(y)
    
    return x, y

def encode_sectors(df, sector_column):
    '''
    Takes in the dataframe and the name of the column that describes the sectors.
    Mutates the dataframe with encoded sector labels and returns an array of the 
    encoded sectors. Required because Keras will not take in labels as strings.
    '''
    
    label_encoder = preprocessing.LabelEncoder()
    df[sector_column] = label_encoder.fit_transform(df[sector_column])
    
    return label_encoder.classes_

In [76]:
df = drop_columns(df, "Date ", "S&P 500 (TR)", "S&P 500 (Net TR)")

In [77]:
df.columns

Index(['S&P 500', 'price return', 'period'], dtype='object')

In [88]:
import sklearn.linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [86]:
encode_sectors(df, 'period')

array([0, 1], dtype=int64)